<a href="https://colab.research.google.com/github/cloudpedagogy/models/blob/main/dl/Siamese_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Siamese Network Model Background

The Siamese neural network is a type of neural network architecture that is primarily used for tasks related to similarity or distance learning. It was introduced by Bromley et al. in 1993 and is designed to compare two inputs and determine their similarity or dissimilarity.

The key idea behind the Siamese network is to have two identical sub-networks (also known as twin networks) that share weights and parameters. These sub-networks take two separate input samples, process them through the same set of layers, and produce feature embeddings. The similarity between the two inputs is then computed based on the distance metric of the feature embeddings.

**Pros of Siamese neural networks**:

1. **Fewer training samples required**: Siamese networks are well-suited for tasks with limited training data, as they learn to compare and distinguish between pairs of samples rather than relying on large amounts of labeled data.

2. **Robust to variations**: Siamese networks are effective when dealing with inputs that have variations, as the shared weights help the network learn common features that are invariant to such variations.

3. **Flexibility**: Siamese networks can be used for various tasks, such as image similarity, signature verification, one-shot learning, and even natural language processing tasks like sentence similarity.

4. **Metric Learning**: Siamese networks inherently learn a similarity metric between inputs, making them suitable for tasks that require a notion of similarity or distance between data points.

**Cons of Siamese neural networks**:

1. **Computational Cost**: Since Siamese networks process two inputs simultaneously, they can be computationally more expensive compared to traditional neural networks that process a single input at a time.

2. **Hyperparameter Tuning**: Siamese networks might require more hyperparameter tuning to achieve optimal performance, especially due to the nature of contrastive loss functions used for training.

3. **Loss Function Design**: Designing an appropriate loss function for the Siamese network can be challenging, and the choice of the loss function can significantly impact the network's performance.

**When to use Siamese neural networks**:

1. **One-shot learning tasks**: When you have limited labeled examples per class and you need to determine similarities between samples, Siamese networks can be a good choice.

2. **Verification and similarity tasks**: Siamese networks are effective for tasks like face verification, signature verification, similarity-based image retrieval, etc.

3. **Metric learning tasks**: When you need to learn a meaningful distance metric between data points, Siamese networks can be used for metric learning applications.

4. **Few-shot learning**: In scenarios where you have only a few labeled examples for each class, Siamese networks can help in learning better representations for comparing and recognizing classes.



# Code Example

In [ ]:
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and reshape the data
train_images = train_images.astype('float32') / 255.
test_images = test_images.astype('float32') / 255.
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))

# Function to generate pairs of similar and dissimilar images
def create_pairs(images, labels):
    num_classes = len(np.unique(labels))
    digit_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    pairs, labels = [], []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1

    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs.append([images[z1], images[z2]])
            inc = np.random.randint(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs.append([images[z1], images[z2]])
            labels += [1, 0]

    return np.array(pairs), np.array(labels)

# Generate pairs for training and testing
train_pairs, train_y = create_pairs(train_images, train_labels)
test_pairs, test_y = create_pairs(test_images, test_labels)

# Siamese Network architecture
def build_siamese_model(input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Shared CNN model
    shared_model = tf.keras.Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
    ])

    output_a = shared_model(input_a)
    output_b = shared_model(input_b)

    # Euclidean distance as similarity measure
    distance = Lambda(lambda x: K.abs(x[0] - x[1]))([output_a, output_b])

    # Final output layer
    output = Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_a, input_b], outputs=output)

    return model

# Build the Siamese Network
input_shape = (28, 28, 1)
siamese_model = build_siamese_model(input_shape)

# Compile the model
siamese_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00006))

# Train the Siamese Network
siamese_model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_y,
                  batch_size=64,
                  epochs=10,
                  validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_y))

# Evaluate the Siamese Network
loss = siamese_model.evaluate([test_pairs[:, 0], test_pairs[:, 1]], test_y)
print("Test Loss:", loss)


# Code breakdown


1. Import the required libraries:
   - `numpy` and `tensorflow` are standard libraries for numerical operations and deep learning, respectively.
   - `Input`, `Flatten`, `Dense`, and `Lambda` are Keras layer classes.
   - `Model` is used to build the Siamese model.
   - `mnist` and `Adam` are used to load the MNIST dataset and configure the Adam optimizer.

2. Load the MNIST dataset:
   - The MNIST dataset consists of grayscale images of handwritten digits from 0 to 9.
   - `mnist.load_data()` loads the training and testing data along with their corresponding labels.

3. Normalize and reshape the data:
   - The pixel values of the images are normalized to the range [0, 1] by dividing by 255.
   - The input images are reshaped to have a single channel (grayscale) and a shape of (28, 28, 1).

4. Define a function to generate pairs of similar and dissimilar images:
   - The function `create_pairs(images, labels)` takes the images and their corresponding labels as input.
   - It creates pairs of images where the first image in the pair is from the same class (similar) and the second image is from a different class (dissimilar).
   - The function returns an array of image pairs and a corresponding binary label array indicating whether the images in each pair are similar (1) or dissimilar (0).

5. Generate pairs for training and testing:
   - The `create_pairs` function is called on the training and testing data to create pairs and labels for training and evaluation.

6. Define the Siamese Network architecture:
   - The function `build_siamese_model(input_shape)` takes the input shape as input and returns a Siamese Neural Network model.
   - It defines two separate input layers for the two images in a pair, followed by a shared CNN model (a simple feedforward neural network).
   - The output of the shared model for both inputs is concatenated and passed through a Lambda layer to calculate the Euclidean distance between the outputs.
   - Finally, a Dense layer with a sigmoid activation function is used to output the probability of the two images being similar.

7. Build the Siamese Network:
   - The `build_siamese_model` function is called with the input shape of the images (28x28x1) to create the Siamese model.

8. Compile the model:
   - The model is compiled with a binary cross-entropy loss function and the Adam optimizer with a learning rate of 0.00006.

9. Train the Siamese Network:
   - The model is trained using the `fit` function with the training data and labels.
   - The `train_pairs[:, 0]` and `train_pairs[:, 1]` slices provide the first and second images of each pair as inputs.
   - The training is performed for 10 epochs with a batch size of 64.

10. Evaluate the Siamese Network:
   - The model is evaluated on the test data using the `evaluate` function.
   - The test pairs and labels are provided as inputs.
   - The test loss is printed to evaluate the model's performance.

The Siamese Network learns to measure the similarity between images and can be useful in applications like face recognition, signature verification, and more, where the objective is to identify similarities between input pairs.

# Real world application

A real-world example of a Siamese Network model in a healthcare setting is its application in medical image analysis, particularly for tasks like medical image similarity and matching. The Siamese Network architecture is commonly used in cases where the goal is to compare two medical images and determine their similarity or dissimilarity for various purposes, such as disease diagnosis, treatment planning, and disease progression tracking.

For instance, in the field of radiology, when a patient undergoes multiple scans (e.g., MRI, CT, or X-ray) over time, it can be valuable to compare these images to assess any changes in the patient's condition. Siamese Networks can be trained to take in two medical images as input and output a similarity score, indicating how similar the images are. The network can be trained with positive pairs (images from the same patient over time) and negative pairs (images from different patients or unrelated time points for the same patient). This way, the model learns to distinguish between images from the same patient and different patients or time points.

The network is typically designed such that the two branches share the same architecture and weights, allowing them to process both images independently before combining the learned representations and making a similarity prediction.

The use of Siamese Networks in healthcare allows for automated and efficient image comparison, aiding medical professionals in diagnosing and monitoring various diseases. It can also help identify disease progression or response to treatment over time.

One of the challenges in applying Siamese Networks in healthcare is acquiring sufficient labeled data for training, as obtaining large annotated medical image datasets can be time-consuming and resource-intensive. Nonetheless, as the field of medical imaging advances, and data becomes more available, Siamese Networks have the potential to play an increasingly important role in enhancing medical diagnosis and patient care.

# FAQ


1. What is a Siamese network?
   A Siamese network is a neural network architecture that consists of two or more identical subnetworks, each sharing the same set of weights and architecture. These subnetworks process different input data and are commonly used for tasks involving similarity or distance comparison.

2. How does a Siamese network work?
   In a Siamese network, the two input samples are passed through the identical subnetworks to obtain their feature representations. These features are then compared using a similarity or distance metric to determine their similarity or dissimilarity.

3. What are the applications of Siamese networks?
   Siamese networks are widely used in tasks such as:
   - Image similarity and verification
   - Signature verification
   - Face recognition and verification
   - Text similarity and paraphrase detection
   - One-shot learning
   - Few-shot learning
   - Metric learning

4. What is the advantage of using Siamese networks over traditional neural networks?
   Siamese networks excel in tasks that require comparing and measuring similarity between inputs. They can learn from limited labeled data and generalize well to unseen data. Additionally, they are effective in learning discriminative features for various tasks without the need for large labeled datasets.

5. Can Siamese networks be used for one-shot or few-shot learning?
   Yes, Siamese networks are well-suited for one-shot and few-shot learning tasks. By learning a similarity metric, they can identify similar items even with very few examples.

6. How are Siamese networks used in image similarity tasks?
   In image similarity tasks, Siamese networks take two images as input and learn to encode them into a shared feature space. The network is trained with pairs of similar and dissimilar images, allowing it to distinguish between similar and dissimilar images effectively.

7. What is the contrastive loss used in Siamese networks?
   Contrastive loss is a common loss function used in Siamese networks. It encourages similar samples to be closer in the feature space while pushing dissimilar samples farther apart. It penalizes the network when the distance between similar samples is large or when the distance between dissimilar samples is small.

8. Are there any real-world applications of Siamese networks?
   Yes, Siamese networks have found applications in various fields, including:
   - Signature verification in banking and finance
   - Face recognition in security systems
   - Text similarity and plagiarism detection in natural language processing
   - Medical image analysis for identifying similar patterns in medical images

9. Can Siamese networks be combined with other architectures?
   Yes, Siamese networks can be combined with other architectures, such as convolutional neural networks (CNNs) for image processing tasks or recurrent neural networks (RNNs) for sequence similarity tasks.

10. How do Siamese networks handle imbalanced datasets?
    Siamese networks naturally handle imbalanced datasets by learning a similarity metric based on pairs of samples rather than individual samples. This allows them to effectively discriminate between classes even with limited data from each class.